In [ ]:
data_path = '../dataset'

In [ ]:
import os

In [ ]:
for path in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, path)):
        print(os.path.join(data_path, path))

In [ ]:
train_data_path = os.path.join(data_path, 'train')
test_data_path = os.path.join(data_path, 'test')

In [ ]:
train_classes = dict()

for path in sorted(os.listdir(train_data_path)):
    if os.path.isdir(os.path.join(train_data_path, path)):
        train_classes.setdefault(len(train_classes), path)
        
train_classes

In [ ]:
test_classes = dict()

for path in sorted(os.listdir(test_data_path)):
    if os.path.isdir(os.path.join(test_data_path, path)):
        test_classes.setdefault(len(test_classes), path)
        
test_classes

In [7]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
torch.cuda.get_device_name(0)

'GeForce GTX 1070'

In [9]:
from torchvision import transforms as T

In [10]:
train_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
val_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
from torchvision.datasets import ImageFolder

In [13]:
train_dataset = ImageFolder(
    root=train_data_path,
    transform=train_transform
)

In [14]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 750
    Root location: ../dataset/train
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [15]:
test_dataset = ImageFolder(
    root=test_data_path,
    transform=val_transform
)

In [16]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 2500
    Root location: ../dataset/test
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [17]:
from torch.utils.data import DataLoader

In [18]:
BATCH_SIZE = 16

In [19]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [20]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=False
)

In [21]:
len(train_loader), len(test_loader)

(47, 157)

In [22]:
import torch.nn as nn
import torch.nn.functional as F

In [23]:
class FoodNet(nn.Module):
    def __init__(self):
        super(FoodNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 8, 3)
        self.conv3 = nn.Conv2d(8, 16, 3)
        self.drop = nn.Dropout(.1)
        self.fc1 = nn.Linear(BATCH_SIZE * 26 * 26, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, BATCH_SIZE * 26 * 26)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = self.fc3(x)
        return x

In [24]:
model = FoodNet()
model = model.to(device)

In [25]:
import torch.optim as optim

In [26]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [27]:
from torch.optim import lr_scheduler

In [28]:
EPOCHS = 25

In [29]:
loaders = {
    'train': train_loader,
    'val': test_loader
}

In [30]:
from copy import deepcopy

In [31]:
%%time

for epoch in range(1, EPOCHS+1):
    running_loss = .0
    running_acc = .0
    best_acc = .0
    best_model = None
    print(f"\nEpoch {epoch}/{EPOCHS}\n{'='*25}")
    for phase in ['train', 'val']:
        if phase == 'train': model.train()
        if phase == 'val': model.eval()
        for inputs, targets in loaders[phase]:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            accuracy = (torch.argmax(outputs, dim=1) == targets).float().mean()

            running_loss += loss.item()
            running_acc += accuracy.item()
        if phase == 'val' and accuracy > best_acc:
            best_acc = accuracy
            best_model = deepcopy(model.state_dict())
        print(f"Loss ({phase}): {running_loss/len(loaders[phase])}, Acc ({phase}): {running_acc/len(loaders[phase])}")


Epoch 1/25
Loss (train): 2.2924972645779875, Acc (train): 0.12310030422312149
Loss (val): 2.9416618590142316, Acc (val): 0.2056414923470491

Epoch 2/25
Loss (train): 2.196672977285182, Acc (train): 0.19604863258118324
Loss (val): 2.8330668453957624, Acc (val): 0.265696087460609

Epoch 3/25
Loss (train): 2.080033233825197, Acc (train): 0.2393617021276596
Loss (val): 2.71517841299628, Acc (val): 0.31488853503184716

Epoch 4/25
Loss (train): 1.9721416945153094, Acc (train): 0.28495440743070966
Loss (val): 2.695035883575488, Acc (val): 0.3197793448996392

Epoch 5/25
Loss (train): 1.9032925925356277, Acc (train): 0.31876899714165546
Loss (val): 2.704692455613689, Acc (val): 0.3243289354500497

Epoch 6/25
Loss (train): 1.8154431023496262, Acc (train): 0.36094224960245985
Loss (val): 2.70630016220603, Acc (val): 0.36720882631411217

Epoch 7/25
Loss (train): 1.5839562086348837, Acc (train): 0.44110942267357034
Loss (val): 2.7578935824382076, Acc (val): 0.3617493176156548

Epoch 8/25
Loss (tra

In [32]:
torch.save(best_model, 'foodnet.pth')